# Illustration of PENCIL data generation

## 1. Generate instances with CoT reasoning steps

The following code generates a dataset of 10,200 Quantified Boolean Formula (QBF) instances with exactly `min_vars = max_vars` variables each. Each instance is augmented with reasoning steps (including special tokens) and forms a very long CoT. The data is saved to disk in JSONL format, where each instance contains the formula text, validity label, and size information.

In [1]:
import os
from dataset_qbf import *
from pencil_utils import *

# Define parameters directly as variables
num_samples = 10200
data_dir = 'data/qbf'
min_vars = 3
max_vars = 3

# Instantiate QBF generator
qbf_gen = QBFGenerator(
    min_vars=min_vars,
    max_vars=max_vars
)

# Generate dataset
data = qbf_gen.generate_dataset(
    num_samples=num_samples,
    data_dir=data_dir,
    format='pencil',
    save=True
)


Generated 1000/10200 samples...
Generated 2000/10200 samples...
Generated 3000/10200 samples...
Generated 4000/10200 samples...
Generated 5000/10200 samples...
Generated 6000/10200 samples...
Generated 7000/10200 samples...
Generated 8000/10200 samples...
Generated 9000/10200 samples...
Generated 10000/10200 samples...


## 2. Build tokenizer and data split

Create a tokenizer by building a vocabulary from all instances, and then convert the text data into tokenized sequences. The tokenizer simply parses the sequence by blank spaces. The meta information of the tokenizer is stored in `meta.pkl`. 

Then, the code splits and saves the processed data as binary files for efficient training, with separate files for training, validation, and test sets. It holds out `val_size` and `test_size` samples as validation and test sets. The remaining samples are treated as the training set. For large-scale online training, the training set could be very large, so the training samples are stored in multiple files, each containing `train_size` samples. During training, we load these files sequentially. For example, the following code will create `train0.bin`, `train1.bin`, `val.bin`, and `test.bin`,

In [2]:
# Build tokenizer, process dataset, etc.
tokenizer = SimpleTokenizer()
tokenizer.build_vocab_from_file(data_dir)
tokenizer.save_meta_to_file(data_dir)

# Train/val splits:
train_size = 5000
val_size = 100
test_size = 100

process_dataset(data_dir, 
                train_size=train_size, 
                val_size=val_size,
                test_size=test_size,
                tokenizer=tokenizer)

Added special token <|startoftext|> with index 38
Added special token <|endofprompt|> with index 17
Added special token <|endoftext|> with index 39

Token to ID mapping:
<UNK>: 0
∃: 1
3: 2
∀: 3
2: 4
1: 5
:: 6
#1: 7
(: 8
∨: 9
¬: 10
): 11
#2: 12
#3: 13
#4: 14
#5: 15
#6: 16
<|endofprompt|>: 17
[CALL]: 18
Question:: 19
prefix_from: 20
Try: 21
=: 22
False: 23
evaluate: 24
Check: 25
#0: 26
True: 27
[SEP]: 28
Answer:: 29
[RETURN]: 30
#7: 31
Formula: 32
#8: 33
#9: 34
#10: 35
#11: 36
#12: 37
<|startoftext|>: 38
<|endoftext|>: 39
Removed existing file: train0.bin
Removed existing file: train1.bin
Processing validation and test sets...
Tokenizing validation and test sets...
Validation set: 100 sequences, 41,517 tokens
Validation max sequence length: 705
Test set: 100 sequences, 41,676 tokens
Test max sequence length: 730
Processing training data in batches...
Processing training batch 1...
Training batch 1 set: 5,000 sequences, 2,143,909 tokens
Training batch 1 max sequence length: 993
Processing

Let's check a training sample

In [3]:
train_bin_file = os.path.join(data_dir, 'train0.bin')
with open(train_bin_file, 'rb') as f:
    train_data = pickle.load(f)
    
sample = train_data[4]
print(f"An Example Sample:")
print(tokenizer.decode(sample))
print(f"------------------------------------") 

An Example Sample:
<|startoftext|> ∃ 2 ∀ 3 ∀ 1 : #1 ( 1 ∨ ¬ 1 ) #2 ( 3 ∨ 1 ∨ ¬ 2 ) #3 ( ¬ 3 ∨ 3 ) #4 ( 2 ) #5 ( 2 ∨ ¬ 2 ) #6 ( ¬ 3 ∨ 3 ) #7 ( ¬ 1 ∨ 1 ) <|endofprompt|> [CALL] Question: prefix_from ∃ 2 Try 2 = False [CALL] Question: prefix_from ∀ 3 Try 3 = False [CALL] Question: prefix_from ∀ 1 Try 1 = False [CALL] Question: evaluate 1 = False 2 = False 3 = False Check #0 ( 1 ∨ ¬ 1 ) True Check #1 ( 3 ∨ 1 ∨ ¬ 2 ) True Check #2 ( ¬ 3 ∨ 3 ) True Check #3 ( 2 ) False [SEP] Answer: False [RETURN] [SEP] Answer: False [RETURN] [SEP] Answer: False [RETURN] Try 2 = True [CALL] Question: prefix_from ∀ 3 Try 3 = False [CALL] Question: prefix_from ∀ 1 Try 1 = False [CALL] Question: evaluate 1 = False 2 = True 3 = False Check #0 ( 1 ∨ ¬ 1 ) True Check #1 ( 3 ∨ 1 ∨ ¬ 2 ) False [SEP] Answer: False [RETURN] [SEP] Answer: False [RETURN] [SEP] Answer: False [RETURN] [SEP] Answer: False [RETURN] <|endoftext|>
------------------------------------


## 3. Training data for PENCIL

We next convert a complete reasoning trace into subsequences. The **masked part** represents the input context that the model sees during training, while the **unmasked part** represents the target output that the model needs to predict. During training, we only calculate loss on the unmasked part.

In [4]:
subseqs = extract_subseqs(
    sample, 
    tokenizer.word2idx[PENCIL_TOKENS['sep']], 
    tokenizer.word2idx[PENCIL_TOKENS['call']], 
    tokenizer.word2idx[PENCIL_TOKENS['return']], 
    tokenizer.word2idx['<|endofprompt|>']
)

for i, subseq in enumerate(subseqs):
    print(f"------------------ Iteration {i} -----------------------") 
    print(f"Generated Subsequence:")
    print(tokenizer.decode(subseq["ids"]))
    print()
    print(f"Masked Part (from previous iteration):")
    print(tokenizer.decode(subseq["ids"][:subseq["mask_idx"]+1]))
    print()
    print(f"Unmasked Part (i.e. tokens to be generated):")
    print(tokenizer.decode(subseq["ids"][subseq["mask_idx"]+1:]))
    print()
    print(f"Reduced Subsequence:")
    reduced_encoding = reduce_encoding(
        subseq["ids"],
        tokenizer.word2idx[PENCIL_TOKENS['sep']], 
        tokenizer.word2idx[PENCIL_TOKENS['call']], 
        tokenizer.word2idx[PENCIL_TOKENS['return']]
    )
    print(tokenizer.decode(reduced_encoding))
    print(f"--------------------------------------------------------\n") 

------------------ Iteration 0 -----------------------
Generated Subsequence:
<|startoftext|> ∃ 2 ∀ 3 ∀ 1 : #1 ( 1 ∨ ¬ 1 ) #2 ( 3 ∨ 1 ∨ ¬ 2 ) #3 ( ¬ 3 ∨ 3 ) #4 ( 2 ) #5 ( 2 ∨ ¬ 2 ) #6 ( ¬ 3 ∨ 3 ) #7 ( ¬ 1 ∨ 1 ) <|endofprompt|> [CALL] Question: prefix_from ∃ 2 Try 2 = False [CALL] Question: prefix_from ∀ 3 Try 3 = False [CALL] Question: prefix_from ∀ 1 Try 1 = False [CALL] Question: evaluate 1 = False 2 = False 3 = False Check #0 ( 1 ∨ ¬ 1 ) True Check #1 ( 3 ∨ 1 ∨ ¬ 2 ) True Check #2 ( ¬ 3 ∨ 3 ) True Check #3 ( 2 ) False [SEP] Answer: False [RETURN]

Masked Part (from previous iteration):
<|startoftext|> ∃ 2 ∀ 3 ∀ 1 : #1 ( 1 ∨ ¬ 1 ) #2 ( 3 ∨ 1 ∨ ¬ 2 ) #3 ( ¬ 3 ∨ 3 ) #4 ( 2 ) #5 ( 2 ∨ ¬ 2 ) #6 ( ¬ 3 ∨ 3 ) #7 ( ¬ 1 ∨ 1 ) <|endofprompt|>

Unmasked Part (i.e. tokens to be generated):
[CALL] Question: prefix_from ∃ 2 Try 2 = False [CALL] Question: prefix_from ∀ 3 Try 3 = False [CALL] Question: prefix_from ∀ 1 Try 1 = False [CALL] Question: evaluate 1 = False 2 = False 3 = False Check #0 ( 1

Apply this procedure to the whole training set, which will create `pencil0.bin` and `pencil1.bin` in the data directory.

In [5]:
process_pencil_data(data_dir, tokenizer)

Converting data to pencil format...

Processing batch 0...
Batch 0 stats:
Samples: 5000 → 55286
Tokens: 2143909 → 6737441

Processing batch 1...
Batch 1 stats:
Samples: 5000 → 55562
Tokens: 2154352 → 6784625

Overall statistics:
Total samples: 10000 → 110848
Total tokens: 4298261 → 13522066
Max length: 993 → 293
Average length: 429.8 → 122.0
Average reductions per sample: 11.1


110848